# Challenge 3

In this challenge we will work on the `Orders.csv` data set in the previous [Subsetting and Descriptive Stats lab](../../lab-subsetting-and-descriptive-stats/your-code/main.ipynb). In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

# Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders.csv` from the "subsetting" lab folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# enter your code here
orders = pd.read_csv("../../lab-subsetting-and-descriptive-stats/your-code/Orders.zip")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [3]:
#pour vérifier que pour un client il n'y a qu'un pays
#orders_c = orders.groupby(['CustomerID','Country'], as_index=False).count()
#orders_c

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [4]:
# your code here
#chaque client se voit attribuer un identifiant ("CustomerID").
#On peut donc dans un premier temps aggréger "amount_spent" à l'aide de la colonne "CustomerID"
orders_sum = orders.groupby('CustomerID', as_index=False).agg({"amount_spent":'sum'})
orders_sum.head()

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40


In [5]:
#Sub Problem 2: How to select customers whose aggregated amount_spent is in a given quantile range?
vip_customers = orders_sum.loc[orders_sum['amount_spent']>orders_sum['amount_spent'].quantile(0.95)]
vip_customers.head()

,CustomerID,amount_spent
0,12346,77183.60
10,12357,6207.67
12,12359,6372.58
50,12409,11072.67
55,12415,124914.53


In [6]:
preferred_customers = orders_sum.loc[(orders_sum['amount_spent'] >orders_sum['amount_spent'].quantile(0.75))&(orders_sum['amount_spent'] < orders_sum['amount_spent'].quantile(0.95))]
preferred_customers.head()

,CustomerID,amount_spent
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
5,12352,2506.04
9,12356,2811.43


In [7]:
#Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?
#orders_sum['customers_type']=np.where(preferred_customers, "Preferred" ,(np.where((vip_customers), "VIP","Normal")))
orders_sum['customers_type']=np.where((orders_sum['amount_spent']>orders_sum['amount_spent'].quantile(0.95)), "Preferred",np.where(((orders_sum['amount_spent'] >orders_sum['amount_spent'].quantile(0.75))&(orders_sum['amount_spent'] < orders_sum['amount_spent'].quantile(0.95))), "VIP","Normal"))
orders_sum.head(10)

,CustomerID,amount_spent,customers_type
0,12346,77183.60,Preferred
1,12347,4310.00,VIP
2,12348,1797.24,VIP
3,12349,1757.55,VIP
4,12350,334.40,Normal
5,12352,2506.04,VIP
6,12353,89.00,Normal
7,12354,1079.40,Normal
8,12355,459.40,Normal
9,12356,2811.43,VIP


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

# Q2: How to identify which country has the most VIP Customers?

# Q3: How to identify which country has the most VIP+Preferred Customers combined?

Provide your solution for Q2 below:

In [8]:
# your code here
# Q2: How to identify which country has the most VIP Customers?
# Dans un premier temps je choisis d'ajouter une colonne 'country' puisque que chaque client a son pays d'associé.Je le fais avec la function .merge()
orders_sum = pd.merge(orders_sum, orders[['CustomerID','Country']], how='inner', left_on = 'CustomerID', right_on = 'CustomerID').drop_duplicates().reset_index(inplace=False)
del orders_sum['index']
orders_sum.head()

,CustomerID,amount_spent,customers_type,Country
0,12346,77183.60,Preferred,United Kingdom
1,12347,4310.00,VIP,Iceland
2,12348,1797.24,VIP,Finland
3,12349,1757.55,VIP,Italy
4,12350,334.40,Normal,Norway


In [9]:
orders_sum['Total_customers']=''
most_vip = orders_sum[['customers_type','Country','Total_customers']].loc[orders_sum['customers_type']=='VIP'].groupby(['customers_type','Country']).count()
most_vip.sort_values('Total_customers', ascending=False)

Total_customers
customers_type Country                         
VIP            United Kingdom               755
               Germany                       29
               France                        20
               Belgium                       11
               Spain                          7
               Switzerland                    6
               Norway                         6
               Italy                          5
               Portugal                       5
               Finland                        4
               Channel Islands                3
               Cyprus                         3
               Austria                        3
               Japan                          2
               Australia                      2
               Israel                         2
               Denmark                        2
               Lebanon                        1
               Malta                          1
               Greece                         1
               Poland                         1
               EIRE                           1
               Sweden                         1
               Canada                         1
               Iceland                        1

In [10]:
# your code here
# Q3: How to identify which country has the most VIP+Preferred Customers combined?
most_vip_pref = orders_sum[['Country','Total_customers']].loc[orders_sum['customers_type']!='Normal'].groupby(['Country']).count()
most_vip_pref.sort_values('Total_customers', ascending=False)

,Total_customers
Country,
United Kingdom,932
Germany,39
France,29
Belgium,12
Switzerland,9
Spain,9
Portugal,7
Norway,7
Italy,5
